In [61]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import librosa
import IPython.display

from src.data import NSynthDataset
from src.models import Autoencoder

In [4]:
model = Autoencoder(h_dim=128)
model.load_state_dict(torch.load('train_results/autoencoder/20200430-175135/model-8000.weights'))

<All keys matched successfully>

In [5]:
loss_fn = nn.BCELoss()

##### Real instrument

In [2]:
val_dataset = NSynthDataset(
    'music-ml-gigioli', 
    'data/nsynth/nsynth-valid', 
    instrument_source=[0], 
    feature_type='mel',
    scaling='normalize'
)

val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
features = val_dataset[0]

In [12]:
features = torch.tensor(features, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
outputs = model(features)

In [19]:
loss_fn(outputs, features).item()

0.0014240677701309323

##### Noise

In [121]:
features = torch.tensor(np.random.rand(1, 1, 128, 251), dtype=torch.float32)
features = (features - features.min()) / (features.max() - features.min())

In [122]:
outputs = model(features)

In [123]:
loss_fn(outputs, features).item()

2.0569937229156494

##### Sample

In [350]:
#sample_file = '269570__vonora__cuckoo-the-nightingale-duet.wav'
#sample_file = '504900__soundmast123__restaurant-ambience.wav'
sample_file = '389684__meggiepie__pots-and-pans.wav'

In [351]:
x, sr = librosa.load(sample_file, sr=16000)

In [352]:
class Sample:
    def __init__(self, sample, sr, n_fft=1024, hop_length=256):
        self.sample = sample
        self.mel = librosa.feature.melspectrogram(y=self.sample, sr=sr, n_fft=n_fft, hop_length=hop_length)
        self.mel = (self.mel - self.mel.min()) / (self.mel.max() - self.mel.min())
        self.mel_len = self.mel.shape[-1]

##### Overlapping segments with masking only on outputs

In [402]:
#window_sizes = [4*sr, 4*sr, 4*sr, 4*sr]
max_window_len = 4*sr
window_scales = [1/16, 1/8, 1/4, 1/2, 1]
step_size = int(sr/2)

samples = []
for i in range(int(len(x)/step_size - int(max_window_len/step_size) + 1)):
    for n, scale in enumerate(window_scales):
        s = Sample(x[i*step_size:i*step_size+max_window_len], sr, n_fft=1024, hop_length=256)
        s.mel_len = int(np.ceil(s.mel_len * scale))
        s.sample = s.sample[:int(len(s.sample) * scale)]
        samples.append(s)

In [407]:
max_mel_length = max([x.mel_len for x in samples])
batch_size = 32

In [408]:
loss_fn = nn.BCELoss(reduction='none')

In [409]:
def mask_tensor(tensor, lens, max_len):
    mask = torch.arange(max_len).expand(len(lens), max_len) < lens.unsqueeze(1)
    mask = mask.unsqueeze(1).unsqueeze(1).expand(tensor.size()).to(torch.float32)
    return tensor*mask

In [410]:
scores = []
for i in range(int(np.ceil(len(samples)/batch_size))):
    batch = samples[i*batch_size:(i+1)*batch_size]
    lens = torch.tensor([x.mel_len for x in batch])
    features = torch.stack([
        torch.tensor(x.mel, dtype=torch.float32) for x in batch
    ]).unsqueeze(1)
    outputs = model(features)
    
    loss = loss_fn(outputs, features)
    loss = mask_tensor(loss, lens, max_mel_length)
    loss = loss.squeeze(1).mean(1).sum(1)/lens
    scores.append(loss.detach().numpy())
scores = np.hstack(scores)
scored_samples = sorted(list(zip(scores, samples)), key=lambda x : x[0])

##### Overlapping segments padded with zeros

In [372]:
window_sizes = [1*sr, 2*sr, 3*sr, 4*sr]
step_size = int(sr/2)

samples = []
for i in range(int(len(x)/step_size - int(window_sizes[-1]/step_size) + 1)):
    for window_size in window_sizes:
        s = Sample(x[i*step_size:i*step_size+window_size], sr, n_fft=1024, hop_length=256)
        samples.append(s)

In [373]:
max_mel_length = max([x.mel_len for x in samples])
batch_size = 32

In [374]:
loss_fn = nn.BCELoss(reduction='none')

In [375]:
def mask_tensor(tensor, lens, max_len):
    mask = torch.arange(max_len).expand(len(lens), max_len) < lens.unsqueeze(1)
    mask = mask.unsqueeze(1).unsqueeze(1).expand(tensor.size()).to(torch.float32)
    return tensor*mask

In [376]:
scores = []
for i in range(int(np.ceil(len(samples)/batch_size))):
    batch = samples[i*batch_size:(i+1)*batch_size]
    lens = torch.tensor([x.mel_len for x in batch])
    features = torch.stack([
        F.pad(torch.tensor(x.mel, dtype=torch.float32), (0, max_mel_length-x.mel_len)) for x in batch
    ]).unsqueeze(1)
    outputs = model(features)
    
    loss = loss_fn(outputs, features)
    loss = mask_tensor(loss, lens, max_mel_length)
    loss = loss.squeeze(1).mean(1).sum(1)/lens
    scores.append(loss.detach().numpy())
scores = np.hstack(scores)
scored_samples = sorted(list(zip(scores, samples)), key=lambda x : x[0])

### Original Source

In [411]:
IPython.display.Audio(x, rate=16000)

### High Scores

In [412]:
IPython.display.Audio(scored_samples[0][1].sample, rate=16000)

In [413]:
IPython.display.Audio(scored_samples[1][1].sample, rate=16000)

In [414]:
IPython.display.Audio(scored_samples[2][1].sample, rate=16000)

In [415]:
IPython.display.Audio(scored_samples[3][1].sample, rate=16000)

In [416]:
IPython.display.Audio(scored_samples[4][1].sample, rate=16000)

In [417]:
IPython.display.Audio(scored_samples[5][1].sample, rate=16000)

In [418]:
IPython.display.Audio(scored_samples[6][1].sample, rate=16000)

In [419]:
IPython.display.Audio(scored_samples[7][1].sample, rate=16000)

In [420]:
IPython.display.Audio(scored_samples[8][1].sample, rate=16000)

### Low Scores

In [421]:
IPython.display.Audio(scored_samples[-1][1].sample, rate=16000)

In [422]:
IPython.display.Audio(scored_samples[-2][1].sample, rate=16000)

In [423]:
IPython.display.Audio(scored_samples[-3][1].sample, rate=16000)

In [424]:
IPython.display.Audio(scored_samples[-4][1].sample, rate=16000)

In [425]:
IPython.display.Audio(scored_samples[-4][1].sample, rate=16000)

In [22]:
scores = []
for i, s in enumerate(samples):
    features = librosa.feature.melspectrogram(y=s, sr=sr, n_fft=1024, hop_length=256)
    features = (features - features.min()) / (features.max() - features.min())
    features = torch.tensor(features, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    outputs = model(features)
    
    loss = loss_fn(outputs, features).item()
    scores.append((i, loss))
scores = sorted(scores, key=lambda x : x[1])

In [27]:
s.min()

-0.036485985

In [11]:
scores

[(13, 0.0007613657508045435),
 (12, 0.0011405345285311341),
 (11, 0.0017651738598942757),
 (7, 0.0018133268458768725),
 (10, 0.002052974421530962),
 (32, 0.00232406216673553),
 (9, 0.0024016001261770725),
 (8, 0.0025628770235925913),
 (31, 0.0027765247505158186),
 (27, 0.002842971356585622),
 (30, 0.002995438175275922),
 (6, 0.0032059955410659313),
 (29, 0.003246042877435684),
 (28, 0.0034053826238960028),
 (3, 0.004472736734896898),
 (26, 0.004718981683254242),
 (14, 0.006228873506188393),
 (2, 0.006462422199547291),
 (24, 0.006846737116575241),
 (1, 0.008014894090592861),
 (15, 0.008051317185163498),
 (41, 0.0098101906478405),
 (0, 0.009924321435391903),
 (39, 0.010106422938406467),
 (40, 0.010764488950371742),
 (23, 0.010869604535400867),
 (34, 0.011729108169674873),
 (25, 0.012726925313472748),
 (33, 0.012835491448640823),
 (38, 0.012888344936072826),
 (5, 0.012927787378430367),
 (4, 0.013349908404052258),
 (35, 0.013758478686213493),
 (20, 0.014595394022762775),
 (22, 0.0146300289

In [14]:
IPython.display.Audio(x, rate=16000)

In [12]:
IPython.display.Audio(x[13*step_size:13*step_size+window_size], rate=16000)